In [1]:
import pandas as pd

In [3]:
import geopandas as gpd

# Load the GeoJSON file
gdf_airspace_ohare_loaded = gpd.read_file("ohare_airspace.geojson")

# Optional: Preview the datag
gdf_airspace_ohare_loaded


,airspace_name,min_altitude,max_altitude,elevation,geometry
0,CHICAGO CLASS B AREA A,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.92530 42.06970 3048.00000, -87..."
1,CHICAGO CLASS B AREA B,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.79310 41.98750 3048.00000, -87..."
2,CHICAGO CLASS B AREA C,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.90470 42.23760 3048.00000, -87..."
3,CHICAGO CLASS B AREA D,3048.00,3048.00,3043.478261,"POLYGON Z ((-88.16470 41.83000 3048.00000, -88..."
4,CHICAGO CLASS B AREA E,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.42280 41.77780 3048.00000, -87..."
5,CHICAGO CLASS B AREA G,3048.00,3048.00,3043.478261,"POLYGON Z ((-88.01280 42.13360 3048.00000, -88..."
6,CHICAGO CLASS B AREA H,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.80000 42.13500 3048.00000, -87..."
7,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.75250 41.86940 1097.28000, -87..."
8,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.55920 41.86920 1097.28000, -87..."
9,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.53500 41.74970 1097.28000, -87..."


In [13]:
# Get bounding box of airspace polygons
minx, miny, maxx, maxy = gdf_airspace.total_bounds
print(f"Airspace Bounds:\nLongitude: {minx} to {maxx}\nLatitude: {miny} to {maxy}")

# Extract lat/lon from geometry
gdf_ohare["longitude"] = gdf_ohare.geometry.x
gdf_ohare["latitude"] = gdf_ohare.geometry.y

# Check which points fall within the map bounds
in_bounds = gdf_ohare[
    (gdf_ohare["longitude"] >= minx) &
    (gdf_ohare["longitude"] <= maxx) &
    (gdf_ohare["latitude"] >= miny) &
    (gdf_ohare["latitude"] <= maxy)
]

print(f"Total points: {len(gdf_ohare)}")
print(f"Points in map view: {len(in_bounds)}")


Airspace Bounds:
Longitude: -89.0613 to -86.6667
Latitude: 40.75 to 43.542


AttributeError: 'Series' object has no attribute 'x'

In [18]:
gdf_ohare.columns

Index(['Unnamed: 0', 'date_of_sighting', 'state', 'city', 'summary',
       'drone_latitude', 'drone_longitude', 'drone_altitude_ft',
       'Facility Name', 'Facility Type', 'Time', 'Time_UTC', 'geometry',
       'index_right', 'airspace_name', 'min_altitude', 'max_altitude',
       'longitude', 'latitude'],
      dtype='object')

In [33]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd

# === Load 3D airspace polygons ===
gdf_airspace = gpd.read_file("ohare_airspace.geojson")
gdf_airspace = gdf_airspace.explode(index_parts=False)

# Convert geometry to coordinates list (with Z)
def polygon_to_coords(poly):
    return [[list(coord) for coord in poly.exterior.coords]]

gdf_airspace["coordinates"] = gdf_airspace["geometry"].apply(polygon_to_coords)

# === Load intrusion points (CSV or GeoDataFrame with lat/lon) ===
# Example intrusion data

# === Create pydeck layers ===
sighting_layer = pdk.Layer(
    "ColumnLayer",
    data=gdf_ohare_clean,
    get_position='[longitude, latitude]',
    get_elevation='alt_scaled',
    elevation_scale=1.2,  # small boost
    radius=300,
    get_fill_color='[255, 255, 0, 180]',
    elevation_range=[1, 10000],  # ensures rendering above base
    extruded=True,
    pickable=True,
    auto_highlight=True
)


# Airspace polygons (3D extruded)
polygon_layer = pdk.Layer(
    "PolygonLayer",
    gdf_airspace,
    get_polygon="coordinates",
    get_fill_color="[200, 30, 0, 100]",
    get_elevation="elevation",
    elevation_scale=1,
    extruded=True,
    pickable=True,
    auto_highlight=True,
)


# === View state (O'Hare coordinates) ===
view_state = pdk.ViewState(
    latitude=41.9742,
    longitude=-87.9073,
    zoom=9,
    pitch=60,
    bearing=0
)

# === Render map ===
r = pdk.Deck(
    layers=[polygon_layer, sighting_layer],
    initial_view_state=view_state,
    tooltip = {
    "html": """
        {% if airspace_name %}
            <b>Airspace:</b> {airspace_name}<br/>
            <b>Altitude Range:</b> {min_altitude}–{max_altitude} ft<br/><br/>
        {% endif %}
        
        {% if summary %}
            <b>Drone Sighting</b><br/>
            <b>Date:</b> {date_of_sighting}<br/>
            <b>Time (UTC):</b> {Time_UTC}<br/>
            <b>City:</b> {city}<br/>
            <b>Summary:</b> {summary}<br/>
            <b>Drone Altitude:</b> {drone_altitude_ft} ft<br/>
        {% endif %}
    """,
    "style": {
        "backgroundColor": "black",
        "color": "white",
        "fontSize": "12px",
        "maxWidth": "300px"
    }
    }
)


# Export to HTML
r.to_html("airspace_intrusions_3d.html")


In [23]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

# Step 1: Load CSV
gdf_ohare = pd.read_csv("ohare2.csv")

# Create geometry from drone_latitude and drone_longitude
gdf_ohare["geometry"] = gdf_ohare.apply(
    lambda row: Point(row["drone_longitude"], row["drone_latitude"]), axis=1
)

# Convert to GeoDataFrame with correct CRS
gdf_ohare = gpd.GeoDataFrame(gdf_ohare, geometry="geometry", crs="EPSG:4326")

# (Optional but good): extract lat/lon for plotting
gdf_ohare["longitude"] = gdf_ohare.geometry.x
gdf_ohare["latitude"] = gdf_ohare.geometry.y


In [27]:
# Drop any rows that can't be visualized
gdf_ohare_clean = gdf_ohare.dropna(subset=["longitude", "latitude", "alt_scaled"])
print(f"Total after cleaning: {len(gdf_ohare_clean)}")  # should be close to 100


Total after cleaning: 165


In [ ]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd

# === Load 3D airspace polygons ===
gdf_airspace = gpd.read_file("ohare_airspace.geojson")
gdf_airspace = gdf_airspace.explode(index_parts=False)

# Convert geometry to coordinates list (with Z)
def polygon_to_coords(poly):
    return [[list(coord) for coord in poly.exterior.coords]]

gdf_airspace["coordinates"] = gdf_airspace["geometry"].apply(polygon_to_coords)

# === Create pydeck layers ===

# Airspace polygons (3D extruded)
polygon_layer = pdk.Layer(
    "PolygonLayer",
    gdf_airspace,
    get_polygon="coordinates",
    get_fill_color="[200, 30, 0, 100]",
    get_elevation="elevation",
    elevation_scale=1,
    extruded=True,
    pickable=True,
    auto_highlight=True,
)

# Drone intrusion dots
dot_layer = pdk.Layer(
    "ScatterplotLayer",
    data=df_intrusions,
    get_position='[longitude, latitude]',
    get_color='[0, 0, 255, 160]',
    get_radius=300,
    pickable=True,
)

# === View state (O'Hare coordinates) ===
view_state = pdk.ViewState(
    latitude=41.9742,
    longitude=-87.9073,
    zoom=9,
    pitch=60,
    bearing=0
)

# === Render map ===
r = pdk.Deck(
    layers=[polygon_layer, dot_layer],
    initial_view_state=view_state,
    tooltip={"text": "{label}"}
)

# Export to HTML
r.to_html("airspace_intrusions_3d.html")


In [8]:
print(df_intrusions.head())


   latitude  longitude  altitude    label
0   41.9802   -87.9085       500  Drone A
1   42.0011   -87.9320      1200  Drone B
2   41.9705   -87.9001       800  Drone C
